In [7]:
import tensorflow as tf
import numpy as np
from keras.models import Sequential
from keras.layers import Dense,LSTM,Dropout
from keras.layers.wrappers import TimeDistributed
import pandas as pd

In [8]:
def dimX(x,ts):
    x=np.asarray(x)
    newX=[]
    for i, c in enumerate(x):
        newX.append([])
        for j in range(ts):
            newX[i].append(c)
    return np.array(newX)

In [9]:
def dimY(Y,ts):
    temp = np.zeros((len(Y), ts, len(chars)), dtype=np.bool)
    for i, c in enumerate(Y):
        for j, s in enumerate(c):
            #print i, j, s
            temp[i, j, char_idx[s]] = 1
    return np.array(temp)

In [10]:
def prediction(preds):
    y_pred=[]
    for i,c in enumerate(preds):
        y_pred.append([])
        for j in c:
            y_pred[i].append(np.argmax(j))
    return np.array(y_pred)

In [11]:
def seq_txt(y_pred):
    newY=[]
    for i,c in enumerate(y_pred):
        newY.append([])
        for j in c:
            newY[i].append(idx_char[j])
    
    return np.array(newY)

In [12]:
def smiles_output(s):
    smiles=np.array([])
    for i in s:
        j=''.join(str(k) for k in i)
        smiles=np.append(smiles,j)
    return smiles

In [13]:
import pandas as pd
data = pd.read_csv('lig_data.csv')
data=data.reindex(np.random.permutation(data.index))
Y=data.SMILES


In [14]:
Y.head()
X=data.ix[:,1:7]
X=X.values
X=X.astype('int')
type(X)

c:\users\gopik\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


numpy.ndarray

In [15]:
maxY=Y.str.len().max()
y=Y.str.ljust(maxY, fillchar='|')

ts=y.str.len().max()

In [16]:
chars = sorted(list( set("".join(y.values.flatten()))))
print('total chars:', len(chars))
char_idx= dict((c, i) for i, c in enumerate(chars))
idx_char = dict((i, c) for i, c in enumerate(chars))

total chars: 25


In [17]:
y_dash=dimY(y,ts)

x_dash=dimX(X,ts)       

   
model = Sequential()
model.add(TimeDistributed(Dense(x_dash.shape[2]), input_shape=(x_dash.shape[1],x_dash.shape[2])))
model.add(LSTM(216, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(216, return_sequences=True))
model.add(TimeDistributed(Dense(y_dash.shape[2], activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer='adam')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [18]:
print(model.input_shape)
print(model.output_shape)

(None, 105, 6)
(None, 105, 25)


In [19]:
model.fit(x_dash,y_dash, nb_epoch=20)

Instructions for updating:
Use tf.cast instead.


c:\users\gopik\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Epoch 1/20
335/335 [==============================] - 7s 22ms/step - loss: 2.4518
Epoch 2/20
335/335 [==============================] - 4s 12ms/step - loss: 1.8368
Epoch 3/20
335/335 [==============================] - 4s 12ms/step - loss: 1.6431
Epoch 4/20
335/335 [==============================] - 4s 11ms/step - loss: 1.5470
Epoch 5/20
335/335 [==============================] - 4s 11ms/step - loss: 1.4665
Epoch 6/20
335/335 [==============================] - 4s 12ms/step - loss: 1.4392
Epoch 7/20
335/335 [==============================] - 4s 11ms/step - loss: 1.4286
Epoch 8/20
335/335 [==============================] - 4s 12ms/step - loss: 1.3971
Epoch 9/20
335/335 [==============================] - 4s 12ms/step - loss: 1.3921
Epoch 10/20
335/335 [==============================] - 4s 11ms/step - loss: 1.3730
Epoch 11/20
335/335 [==============================] - 4s 12ms/step - loss: 1.3934
Epoch 12/20
335/335 [==============================] - 4s 11ms/step - loss: 1.3895
Epoch 13/20
3

In [ ]:
x_pred=[[0,0,0,1,0,0],
        [0,1,0,0,0,0],
        [0,0,0,0,0,1],[1,0,0,0,0,0]]
x_pred=dimX(x_pred,ts)      
preds=model.predict(x_pred)
y_pred=prediction(preds)
y_pred=seq_txt(y_pred)
s=smiles_output(y_pred)
print(s)